In [30]:
import numpy as np

shape = (100, 50)
probability = (0.4, 0.3)

# STEP 1 -- generate random maze
# generate matrices representing whether a square has left, right, top, or bottom
vertical = np.random.choice(a=[True, False], size=(shape[0]+1, shape[1]+1), p=[probability[0], 1-probability[0]])
horizontal = np.random.choice(a=[True, False], size=(shape[0]+1, shape[1]+1), p=[probability[1], 1-probability[1]])

# setup outermost walls
vertical[:, 0] = True
vertical[:, -1] = True
horizontal[0, :] = True
horizontal[-1, :] = True

In [31]:
# STEP 1.1 -- display random maze
from PIL import Image, ImageDraw
cell_size = (10, 10)
image_size = (shape[0]*cell_size[0]+shape[0]+1, shape[1]*cell_size[1]+shape[1]+1)

# generate 2D matrix of cells
image_arr = np.zeros(shape=image_size, dtype=bool)
# fill in horizontal lines
for row in range(horizontal.shape[0]):
    for line in range(horizontal.shape[1]):
        line_start = line*(cell_size[1]+1*(line!=0))
        line_end = line_start + cell_size[1] + 1
        image_arr[row*(cell_size[0]+1*(row!=0)), line_start:line_end] = horizontal[row, line]

# fill in vertical lines
for col in range(vertical.shape[1]):
    for line in range(horizontal.shape[0]):
        line_start = line*(cell_size[1]+1*(line!=0))
        line_end = line_start + cell_size[1]+1
        image_arr[line_start:line_end, col*(cell_size[0]+1*(col!=0))] = \
            image_arr[line_start:line_end, col*(cell_size[0]+1*(col!=0))] | vertical[line, col]

maze = Image.fromarray(image_arr)
# maze.show()

# successfully generated a maze

In [32]:
# STEP 2 -- reduce to graph problem
# generate adjacency list for graph representation
adjacency_list = []
count = 0
for i in range(shape[0]):
    for j in range(shape[1]):
        adjacency_list.append([])
        if horizontal[i, j] == 0:
            adjacency_list[count].append(count-shape[1])
        if vertical[i, j] == 0:
            adjacency_list[count].append(count-1)
        if vertical[i, j+1] == 0:
            adjacency_list[count].append(count+1)
        if horizontal[i+1,j] == 0:
            adjacency_list[count].append(count+shape[1])
        count += 1

In [33]:
# list of nodes defining th ground
ground_nodes = [val for val in range(shape[0]*shape[1]-shape[1], shape[0]*shape[1])]

In [34]:
# STEP 2.1 -- solve as graph traversal problem
import sys
from operator import itemgetter

def shortest_path(adj_list, start_vertex, destination_nodes):
    dist = [sys.maxsize-1]*(shape[0]*shape[1])
    dist[start_vertex] = 0

    visited = [False]*(shape[0]*shape[1])
    visited[0] = True

    prev = [-1]*(shape[0]*shape[1])
    q = []
    q.append(start_vertex)
    found = False

    while q:
        v = q.pop(0)
        for i in adj_list[v]:
            if not visited[i]:
                visited[i] = True
                dist[i] = dist[v]+1
                prev[i] = v
                q.append(i)

                if i in destination_nodes:
                    found = True
                    break
            if found:
                break

    if found:
        # figure out which ground vertex was reached first
        reached = [v for v in ground_nodes if prev[v] != -1]

        # select random ground
        if len(reached) > 1:
            ground = reached[np.random.randint(0, len(reached)-1)]
        else:
            ground = reached[0]

        path = []
        path.append(ground)
        node = ground

        while prev[node] != -1:
            path.append(prev[node])
            node = prev[node]

        path.reverse()
        return path
    else:
        print('Path NOT FOUND')
        return []

path = shortest_path(adjacency_list, 0, destination_nodes=ground_nodes)

In [38]:
import copy
# create a function to colour the cell
def colour_cell(cell_num, image_arr):
    hor_location = (cell_num % shape[1])*(cell_size[1] + 1) + 1
    ver_location = int(cell_num / shape[1])*(cell_size[0] + 1) + 1
    image_arr[ver_location:ver_location+cell_size[0]+1, hor_location:hor_location+cell_size[1]+1] = 1

    return image_arr


def animate_path(path, og_maze):
    frames = []
    frame = og_maze
    frames.append(Image.fromarray(frame))
    for cell in path:
        frame = colour_cell(cell, frame)
        frames.append(Image.fromarray(frame).copy())
    frames[0].save("out.gif", save_all=True, append_images=frames[1:], duration=10)

animate_path(path, image_arr)
print('Done')

Done
